## Measuring VNC length

The VNC length is measured using a centerline estimation.
The steps are as follows:

1. The image is binarized, so we can apply a distance transform.
The distance transform is very succeptible to holes and concavities, so we also remove holes from the binary image and apply a binary opening to smooth edges.
2. We apply a 'chessboard' distance transform (minimum difference between two points along any coordinate dimension) to the binary image, which naturally results in peaks over the centerline of the binary image.
Given the embryo morphology, we expect several center points along the anterior-posterior axis and eventually another (smaller) set of points towards the brain lobes.
3. To separate these points, we fit a RANSAC model that, given the majority of the points are along the AP axis, fits a line along the VNC centerline.
4. From this line intersection to the binary image, we are able to measure the VNC length.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import matplotlib.pyplot as plt
from tifffile import imread

from snazzy_processing import centerline, find_hatching, utils, vnc_length

experiment_name = '20250828'
root_dir = Path.cwd().parent
project_dir = root_dir.joinpath('data', experiment_name)
print(f"Selected data for experiment: {experiment_name}")

The cell below shows the different binary images from each method and the different VNC length measurements.
It also illustrates how the VNC length is calculated.
Visualize centerline method for a single frame. Includes ROI, inliers (green), outliers(red), and centerline.

Change the `frame_number` variable to change the frame that is displayed for a given embryo.
Change `i` to select a different embryo.

The `threshold_method` used to binarize the image has a big impact in the VNC length measurement.
The entire method is based on measuring a binary image, so if that binary image is not representative of the actual VNC, the length also won't be.
The two options here are `otsu` and `multiotsu`.
Use the default multiotsu for most movies and if a higher threshold is needed (because the image has low signal intensity) change the method to `otsu`.

In [ ]:
i = 0
frame_number = 500

img_dir = project_dir.joinpath("embs")
imgs = sorted(img_dir.glob("*ch2.tif"), key=utils.emb_number)

img_path = imgs[i]
img = imread(img_path, key=frame_number)

binary_image = centerline.binarize(img, threshold_method="otsu")

fig, axes = plt.subplots(2, 3, figsize=(16, 3))

axes[0][0].imshow(img, cmap="viridis", vmin=0.05)
axes[0][1].imshow(binary_image, cmap="viridis", vmin=0.05)
centerline.view_centerline_dist(binary_image, ax=axes[0][2], thres_rel=0.5, min_dist=7)

binary_image_multiotsu = centerline.binarize(img)
axes[1][0].imshow(img, cmap="viridis", vmin=0.05)
axes[1][1].imshow(binary_image_multiotsu, cmap="viridis", vmin=0.05)
centerline.view_centerline_dist(
    binary_image_multiotsu, ax=axes[1][2], thres_rel=0.5, min_dist=7
)

for row in axes:
    for ax in row:
        ax.set_xticks([])
        ax.set_yticks([])

In [ ]:
i = 0
frame_number = 600

img_dir = project_dir.joinpath("embs")
imgs = sorted(img_dir.glob("*ch2.tif"), key=utils.emb_number)

img_path = imgs[i]
img = imread(img_path, key=frame_number)
binary_img = centerline.binarize(img, threshold_method="otsu")
dt_img = centerline.get_DT_image(binary_img)

fig, ax = plt.subplots()
ax.set_axis_off()
ax.imshow(dt_img)

Plot VNC length for a single embryo.

Change `i` to select a different embryo.

In [ ]:
i = 0
interval = 20

img_dir = project_dir.joinpath("embs")
embs = sorted(img_dir.glob("*-ch2.tif"), key=utils.emb_number)
emb = embs[i]

hp = find_hatching.find_hatching_point(emb)
hp -= hp % interval

img = imread(emb, key=range(0, hp, interval))
vnc_len = vnc_length.measure_VNC_centerline(img)
x = list(range(0, len(vnc_len) * interval, interval))

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.suptitle(f"VNC length estimation - {emb.stem}")

ax.plot(x, vnc_len)